In [36]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold
import sklearn.metrics as metrics
#from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
#from sklearn.decomposition import NMF

In [2]:
CBR = CatBoostRegressor()

In [82]:
# Read in the training file as RATINGS
ratings = pd.read_csv('data/training.csv')
# Convert timestamp to datetime
ratings['timestamp'] = pd.to_datetime(ratings['timestamp'], unit='s')

In [83]:
#READ MOVIE INFO
bow = pd.read_pickle('data/fuller_df.pkl')

In [84]:
bow['bag_of_words'].fillna('', inplace=True)
bow['runtime'].fillna(bow.runtime.mean(), inplace=True)
bow['english'].fillna(0, inplace=True)

In [85]:
'''bow['big_bag'] = bow['genre'].apply(lambda ls: ' '.join(ls)) + ' ' + bow['bag_of_words']
bow['big_bag'] = bow['big_bag'].apply(lambda s: ' '.join(s.split()[:7]))'''
bow = bow.drop(columns=['genre', 'bag_of_words'])

In [86]:
bow

,movie_id,year,title,runtime,english
0,1,1995,Toy Story,81.0,1.0
1,2,1995,Jumanji,104.0,1.0
2,3,1995,Grumpier Old Men,101.0,1.0
3,4,1995,Waiting to Exhale,127.0,1.0
4,5,1995,Father of the Bride Part II,106.0,1.0
...,...,...,...,...,...
6793,3948,2000,Meet the Parents,108.0,1.0
6794,3949,2000,Requiem for a Dream,102.0,1.0
6795,3950,2000,Tigerland,101.0,1.0
6796,3951,2000,Two Family House,104.0,1.0


In [87]:
# Load in USERS
users = pd.read_table('data/users.dat', delimiter='::', header=None, 
                      names=['user', 'sex', 'age', 'occupation', 'zip_code'], 
                      engine='python')
users['zip_code'] = users['zip_code'].apply(lambda s: int(s[:5]))

In [88]:
ratings = ratings.merge(users, left_on='user', right_on='user')

In [89]:
#combine the 2
ratings = bow.merge(ratings, left_on='movie_id', right_on='movie')

In [90]:
ratings = ratings.sort_values(by='timestamp').reset_index(drop=True)
ratings.drop(columns=['title', 'movie', 'timestamp'], inplace=True)

In [91]:
ratings

,movie_id,year,runtime,english,user,rating,sex,age,occupation,zip_code
0,858,1972,175.000000,1.0,6040,4,M,25,6,11106
1,593,1991,119.000000,1.0,6040,5,M,25,6,11106
2,2384,1998,92.000000,1.0,6040,4,M,25,6,11106
3,2019,1954,104.134745,0.0,6040,5,M,25,6,11106
4,1961,1988,133.000000,1.0,6040,4,M,25,6,11106
...,...,...,...,...,...,...,...,...,...,...
799995,1259,1986,89.000000,1.0,1875,3,M,35,12,94107
799996,589,1991,137.000000,1.0,1875,4,M,35,12,94107
799997,2948,1963,115.000000,1.0,1875,5,M,35,12,94107
799998,110,1995,177.000000,1.0,1875,4,M,35,12,94107


In [31]:
# Create a TRAINING and VALIDATION set. validation set will be the
# last 20% of the training set, ordered by date
training = ratings.loc[:len(ratings)*.8-1, :].copy()
validation = ratings.loc[len(ratings)*.8:, :].copy()

In [32]:
y_train = training.pop('rating')
y_test = validation.pop('rating')

In [33]:
CBR = CatBoostRegressor(cat_features=['movie_id', 'user', 'sex'])
#train the model
CBR.fit(training, y_train) 

In [35]:
# make the prediction using the resulting model
#preds_class = CBR.predict(validation, prediction_type='Class')
preds_proba = CBR.predict(validation, prediction_type='Probability')
preds_raw_vals = CBR.predict(validation, prediction_type='RawFormulaVal')
#print("Class", preds_class)
print("Proba", preds_proba)
print("Raw", preds_raw_vals)

Proba [[0.04727303 0.95272697]
 [0.01071047 0.98928953]
 [0.00899489 0.99100511]
 ...
 [0.01824713 0.98175287]
 [0.02112843 0.97887157]
 [0.01873941 0.98126059]]
Raw [3.0033884  4.52576572 4.70206352 ... 3.98533174 3.83578083 3.95820936]


In [44]:
metrics.mean_squared_error(y_test.values, preds_raw_vals, squared=False)

0.9966226010006334

In [92]:
requests = pd.read_csv('data/requests.csv')

In [93]:
user_req = pd.merge(requests, users,  how='left', left_on='user', right_on = 'user')
user_req = user_req.merge(bow.drop(columns='title'), how='left', left_on='movie', right_on='movie_id')

In [57]:
CBR = CatBoostRegressor(cat_features=['movie_id', 'user', 'sex'])

,user,sex,age,occupation,zip_code
4957,4958,M,18,7,55403


In [94]:
ratings.rename(columns={'movie_id':'movie'}, inplace=True)
ratings = ratings[['user', 'movie', 'sex', 'age', 'occupation', 'zip_code', 'year', 'runtime', 'english', 'rating']]

In [100]:
user_req

,user,movie,sex,age,occupation,zip_code,year,runtime,english
0,4958,1924,M,18,7,55403,1958,79.0,1.0
1,4958,3264,M,18,7,55403,1992,86.0,1.0
2,4958,2634,M,18,7,55403,1959,124.0,1.0
3,4958,1407,M,18,7,55403,1996,111.0,1.0
4,4958,2399,M,18,7,55403,1985,108.0,1.0
...,...,...,...,...,...,...,...,...,...
200204,1875,3793,M,35,12,94107,2000,104.0,1.0
200205,1875,2160,M,35,12,94107,1968,136.0,1.0
200206,1875,1035,M,35,12,94107,1965,174.0,1.0
200207,1875,1580,M,35,12,94107,1997,98.0,1.0


In [97]:
rates = ratings.pop('rating')

In [113]:
CBRF = CatBoostRegressor(cat_features=['user', 'movie', 'sex', 'zip_code', 'year', 'runtime', 'english'])

CBRF.fit(ratings, rates)

Learning rate set to 0.128278
0:	learn: 1.0843772	total: 399ms	remaining: 6m 39s
1:	learn: 1.0558100	total: 996ms	remaining: 8m 17s
2:	learn: 1.0331635	total: 1.48s	remaining: 8m 12s
3:	learn: 1.0153768	total: 1.78s	remaining: 7m 23s
4:	learn: 1.0009199	total: 1.94s	remaining: 6m 25s
5:	learn: 0.9892850	total: 2.08s	remaining: 5m 45s
6:	learn: 0.9803296	total: 2.38s	remaining: 5m 37s
7:	learn: 0.9731237	total: 2.65s	remaining: 5m 28s
8:	learn: 0.9674097	total: 3.02s	remaining: 5m 32s
9:	learn: 0.9630226	total: 3.29s	remaining: 5m 25s
10:	learn: 0.9594997	total: 3.43s	remaining: 5m 8s
11:	learn: 0.9567049	total: 3.71s	remaining: 5m 5s
12:	learn: 0.9544419	total: 3.85s	remaining: 4m 52s
13:	learn: 0.9526704	total: 3.99s	remaining: 4m 40s
14:	learn: 0.9511911	total: 4.13s	remaining: 4m 30s
15:	learn: 0.9499486	total: 4.28s	remaining: 4m 23s
16:	learn: 0.9487904	total: 4.61s	remaining: 4m 26s
17:	learn: 0.9479393	total: 4.88s	remaining: 4m 26s
18:	learn: 0.9470544	total: 5.03s	remaining: 4

In [114]:
scores = CBRF.predict(user_req, prediction_type='RawFormulaVal')

In [118]:
requests['rating'] = scores

In [120]:
requests.to_csv('regression.csv', index=False)

In [122]:
ratings['rating'] = rates

In [124]:
ratings.to_pickle('ratingsandinfo.pkl')